In [1]:
import tensorflow as tf
import tensorflow_gan as tfgan
import numpy as np
from numpy import zeros
from numpy import ones
from numpy import hstack
from numpy.random import rand
from numpy.random import randn
import pandas as pd
import glob
import os
import PIL
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model
import seaborn as sb
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pydot
sb.set()

In [2]:
import os 
import earthpy as et 

In [3]:
## Code needed to set a new working directory

#my_path = os.path.join(et.io.HOME, 'recommender_system')
#os.mkdir(my_path)

## Set Working Directory
os.chdir(os.path.join(et.io.HOME, 'recommender_system'))

In [4]:
# Check if it worked
os.getcwd()

'/Users/chriskamerin/recommender_system'

In [5]:
df =pd.read_csv('data_in_use/headphones2hp.csv')

In [6]:
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
df['user_id']  = pd.Categorical(df.user_id).codes
df['headphone_name']  = pd.Categorical(df.headphone_name).codes

In [8]:
#Set columns we want to normalize

cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance','noise_canceling_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover', 'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
#       'price_sensitivity'
]
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']



In [9]:
df =df[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover','attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
      'weighted_harmonic_distortion', 'soundstage', 'imaging',
      'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
      'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]
#ydf =df[[ 'attached_mic', 'recording_quality',
#       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
#      'weighted_harmonic_distortion', 'soundstage', 'imaging',
#      'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
#       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]
#xtest=df1[['game_use', 'rec_use', 'studio_use', 'exercise_use',
#       'office_use', 'podcast_use', 'travel_use', 'other_use',
#       'sound_quality_importance', 'positional_importance', 'loud_importance',
#       'mic_importance', 'wireless_importance', 'in_ear_importance',
#       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][12:]
#ytest=df1[[ 'attached_mic', 'recording_quality',
#       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
#       'weighted_harmonic_distortion', 'soundstage', 'imaging',
#       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
#       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][12:]

In [10]:
#normalize columns in df

df=df.fillna(0)
df[imp_col] = df[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df[price_col] = df[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df[hp_feat_col] = df[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))   
df[remaining_col] = df[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))       
#df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [12]:
# define the standalone discriminator model
def define_discriminator(n_inputs=34):
	model = Sequential()
	model.add(Dense(256, kernel_initializer='he_uniform', input_dim=n_inputs))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(256, kernel_initializer='he_uniform'))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(34, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
 
# define the standalone generator model
def define_generator(latent_dim, n_outputs=34):
	model = Sequential()
	model.add(Dense(256, kernel_initializer='he_uniform', input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(256, kernel_initializer='he_uniform'))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(n_outputs, activation='linear'))
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model
 
# generate n real samples with class labels
def generate_real_samples(n):
	# generate inputs in [-0.5, 0.5]
	X1 = rand(n) - 0.5
	# generate outputs X^2
	X2 = X1 * X1
	# stack arrays
	X1 = X1.reshape(n, 1)
	X2 = X2.reshape(n, 1)
	X = hstack((X1, X2))
	# generate class labels
	y = ones((1, n))
	return  y
#   return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
	# generate points in the latent space
	x_input = randn(latent_dim * n)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 34))
	return X, y
	
	
# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=34):
	# prepare real samples
	# x_real, y_real = generate_real_samples(n)
	x_real, y_real = df, generate_real_samples(n)
	# evaluate discriminator on real examples
	_, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
	# evaluate discriminator on fake examples
	_, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print(epoch, acc_real, acc_fake)
	# scatter plot real and fake data points
	#pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
	#pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
	#pyplot.show()
 
gen_x = []
gen_y = []
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=16, n_batch=68, n_eval=1):
	# determine half the size of one batch, for updating the discriminator
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		# x_real, y_real = generate_real_samples(half_batch)
		x_real, y_real = df[i:i+1], generate_real_samples(half_batch)
		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		gen_x.append(x_fake)
		gen_y.append(y_fake)
		# update discriminator
		d_model.train_on_batch(x_real, y_real)
		d_model.train_on_batch(x_fake, y_fake)
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 34))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)
		# evaluate the model every n_eval epochs
		if (i+1) % n_eval == 0:
			summarize_performance(i, g_model, d_model, latent_dim)
 
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

ValueError: Data cardinality is ambiguous:
  x sizes: 16
  y sizes: 1
Please provide data which shares the same first dimension.